In [1]:
from VisionEngine.datasets import guppies
from VisionEngine.utils.config import process_config
from VisionEngine.utils import factory
import sys
import os
from PIL import Image
from itertools import product
from dotenv import load_dotenv
from pathlib import Path

import numpy as np
import tensorflow as tf

import tqdm
import warnings
import numba
import pickle
import random

In [2]:
seed_value = 42
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)

In [3]:
GPU = 0

In [4]:
generations = 1000
n_latents = 4
latent_size = 10

In [5]:
config_file = "/home/etheredge/Workspace/VisionEngine/VisionEngine/configs/guppy_nouveau_finetune_config.json"
config = process_config(config_file)
checkpoint_path = "/home/etheredge/Workspace/VisionEngine/checkpoints/guppy_nouveau_finetune/2020-230-11/guppy_nouveau_finetune.hdf5"

In [6]:
env_path = Path('../') / '.env'
load_dotenv(dotenv_path=env_path)

True

In [7]:
config.data_loader.use_generated = False
config.data_loader.use_real = True

config.data_loader.shuffle = True

In [8]:
data_loader = factory.create(
            "VisionEngine.data_loaders."+config.data_loader.name
            )(config)

importing VisionEngine.data_loaders.vae_data_loader
getattr DataLoader


In [9]:
with tf.device(f"GPU:{GPU}"):
    model = factory.create(
                "VisionEngine.models."+config.model.name
                )(config)
    model.load(checkpoint_path)

importing VisionEngine.models.nouveau_vae_model
getattr VAEModel
Model: "vlae"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
noise_layer (Sequential)        (None, 256, 256, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
encoder (Model)                 [(None, 524288), (No 38380171    noise_layer[0][0]                
__________________________________________________________________________________________________
variational_layer (VariationalL (None, 10)           5242890     encoder[1][0]                    
______________________________

In [10]:
def embed_images(x):
    outputs = [
        model.model.get_layer('variational_layer').output,
        model.model.get_layer('variational_layer_1').output,
        model.model.get_layer('variational_layer_2').output,
        model.model.get_layer('variational_layer_3').output
    ]
    encoder = tf.keras.Model(model.model.inputs, outputs)
    return encoder.predict(x)

In [11]:
with tf.device(f"GPU:{GPU}"):
    data_loader = factory.create(
            "VisionEngine.data_loaders."+config.data_loader.name
            )(config)
    parents = embed_images(data_loader.get_test_data())

importing VisionEngine.data_loaders.vae_data_loader
getattr DataLoader


In [12]:
with tf.device(f'/device:GPU:{GPU}'):
    parents = tf.transpose(tf.Variable(parents), (1,0,2))
    parents = parents[:900]

In [35]:
with tf.device(f"GPU:{GPU}"):
    POPULATION_SIZE = 1000
    BATCH_SIZE = 100
    N_GENERATIONS = 1000
    mutation_per = 0.1
    parent_record = []
    persistence=0.5
    temperature=0.2

    orange_min = tf.constant([0.9, 0.55, 0.])
    orange_min = tf.stack([tf.fill((BATCH_SIZE,256,256), orange_min[0]),
              tf.fill((BATCH_SIZE,256,256), orange_min[1]),
              tf.fill((BATCH_SIZE,256,256), orange_min[2])],
              axis=-1)
    orange_max = tf.constant([1., 0.75, 0.1])
    orange_max = tf.stack([tf.fill((BATCH_SIZE,256,256), orange_max[0]),
              tf.fill((BATCH_SIZE,256,256), orange_max[1]),
              tf.fill((BATCH_SIZE,256,256), orange_max[2])],
              axis=-1)
    black_min = tf.constant([0., 0., 0., 0.8])
    black_min = tf.stack([tf.fill((BATCH_SIZE,256,256), black_min[0]),
              tf.fill((BATCH_SIZE,256,256), black_min[1]),
              tf.fill((BATCH_SIZE,256,256), black_min[2])],
              axis=-1)
    black_max = tf.constant([0.2, 0.2, 0.2])
    black_max = tf.stack([tf.fill((BATCH_SIZE,256,256), black_max[0]),
              tf.fill((BATCH_SIZE,256,256), black_max[1]),
              tf.fill((BATCH_SIZE,256,256), black_max[2])],
              axis=-1)

    weights = [1., 1.]

    for _ in tqdm.tqdm_notebook(range(N_GENERATIONS), desc='generation'):
        fitness = []
        ds = tf.data.Dataset.from_tensor_slices(parents).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
        for batch in tqdm.tqdm_notebook(ds, leave=False, desc='fitness loop'):
            batch = tf.reshape(batch, (4, BATCH_SIZE, 10))
            x_hat = model.decoder([batch[0], batch[1], batch[2], batch[3]])
            orange_vals = tf.math.logical_and(
                    tf.math.greater(x_hat,orange_min),
                    tf.math.less(x_hat, orange_max))
            percent_orange = tf.math.divide(
                tf.reduce_sum(tf.cast(tf.reduce_all(
                        orange_vals,axis=(3)
                    ),dtype=tf.float32
                ),axis=(1,2)),
                256*256)
            black_vals = tf.math.logical_and(
                    tf.math.greater(x_hat,black_min),
                    tf.math.less(x_hat, black_max))
            percent_black = tf.math.divide(
                tf.reduce_sum(tf.cast(tf.reduce_all(
                        black_vals,axis=(3)
                    ),dtype=tf.float32
                ),axis=(1,2)),
                256*256)
            # fitness is just a simple weighted sum here
            fit = tf.math.reduce_sum([percent_orange*weights[0],percent_black*weights[1]],axis=0)
            fitness.extend(fit)

        ####### survivors
        p_dist = tf.argsort(fitness)
        indexes = tf.range(len(parents))
        
#         indices_p = np.random.choice(indexes, int(POPULATION_SIZE*persistence), p=p_dist)
        indices_p = [[i.numpy()] * p_dist[i].numpy() for i in indexes]
        indices_p = [item for sublist in indices_p for item in sublist]
        indices_p = list(tf.random.shuffle(indices_p).numpy())[:int(POPULATION_SIZE*persistence)]
        survivors_p = tf.gather(parents, indices_p)
        
#         indices_t = np.random.choice(indexes, int(POPULATION_SIZE*temperature))
        indices_t = list(tf.random.shuffle(indexes).numpy())[:int(POPULATION_SIZE*temperature)]
        survivors_t = tf.gather(parents, indices_t)
        
        survivors = tf.concat([survivors_p, survivors_t], axis=0)
        
        
        ####### offspring
        # pick a couple of parents
        parent1 = [survivors[i] for i in 
            tf.random.uniform([POPULATION_SIZE - len(survivors),], minval=0, maxval=len(survivors),
                              dtype=tf.dtypes.int32)
        ]

        parent2 = [survivors[i] for i in 
            tf.random.uniform([POPULATION_SIZE - len(survivors),], minval=0, maxval=len(survivors),
                              dtype=tf.dtypes.int32)
        ]

        # randomly combine traits from each parent with equal probability
        mask = tf.dtypes.cast(tf.random.uniform([POPULATION_SIZE - len(survivors),4,10], minval=0, maxval=2,
                              dtype=tf.dtypes.int32), tf.bool)
        
        children = tf.where(mask, parent1, parent2)
        
        # randomly mutate some loci
        ix = list(tf.random.shuffle([i for i in tf.range(len(children))]).numpy())[:int(len(children)*mutation_per)]
        indices = tf.stack([
            tf.constant([[
                i,
                tf.random.uniform([1,], minval=0, maxval=5, dtype=tf.dtypes.int32).numpy()[0],
                tf.random.uniform([1,], minval=0, maxval=11, dtype=tf.dtypes.int32).numpy()[0],
            ]]) 
            for i in ix
        ])
        
        values = tf.stack([  
            tf.random.uniform([1,], minval=tf.math.reduce_min(parents), maxval=tf.math.reduce_max(parents), dtype=tf.dtypes.float32)
            for i in ix
        ])


        children = tf.tensor_scatter_nd_update(children, indices, values)
        
        parents = tf.concat([survivors, children], axis=0)
        
        parent_record.append(parents)


/home/etheredge/miniconda3/envs/VisionEngine/lib/python3.7/site-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/home/etheredge/miniconda3/envs/VisionEngine/lib/python3.7/site-packages/ipykernel_launcher.py:36: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [36]:
with open("parent_record_2.p", "wb") as f:
    pickle.dump(parent_record, f)